# Ejercicio Clustering. Detección de Anomalías

Uno de los usos de los algoritmos de clustering es la Detección de Anomalías, esto es, la detección de observaciones anómalas, aquellas que no siguen un comportamiento normal. Si el objetivo del clustering es encontrar grupos de elementos similares, aquellos elementos que no son similares a ningún grupo se pueden considerar como elementos anómalos.

Para este ejercicio vamos a usar un [Dataset de transacciones de tarjetas de crédito](https://www.kaggle.com/arjunbhasin2013/ccdata), donde cada observacion es un cliente distinto.

Nuestro objetivo es implementar un modelo que agrupa las transacciones apropiadamente y encontrar los potenciales outliers, es decir, aquellas transacciones que son sospechosas de ser un fraude o un error. Para resolver este ejercicio correctamente hay que investigar, en vez de simplemente seguir a rajatabla lo enseñado en el curso.

**Pistas:**

- Hemos explicado un algoritmo de clustering que no solo asigna elementos a clusters válidos, sino que también clasifica elementos como valores extremos (outliers). 

- Para la búsqueda de hiperparámetros, un buen sitio para mirar es [ParameterSampler](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html).

In [7]:
import pandas as pd
df = pd.read_csv("D:/datasets/Curso_Mauel_Garrido/CC GENERAL.csv")

In [8]:
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [9]:
df.shape

(8950, 18)

In [10]:
df.dtypes

CUST_ID                              object
BALANCE                             float64
BALANCE_FREQUENCY                   float64
PURCHASES                           float64
ONEOFF_PURCHASES                    float64
INSTALLMENTS_PURCHASES              float64
CASH_ADVANCE                        float64
PURCHASES_FREQUENCY                 float64
ONEOFF_PURCHASES_FREQUENCY          float64
PURCHASES_INSTALLMENTS_FREQUENCY    float64
CASH_ADVANCE_FREQUENCY              float64
CASH_ADVANCE_TRX                      int64
PURCHASES_TRX                         int64
CREDIT_LIMIT                        float64
PAYMENTS                            float64
MINIMUM_PAYMENTS                    float64
PRC_FULL_PAYMENT                    float64
TENURE                                int64
dtype: object

In [11]:
customer_ids = df.CUST_ID
df = df.drop(columns = "CUST_ID")

In [12]:
df.columns[df.isnull().any()]

Index(['CREDIT_LIMIT', 'MINIMUM_PAYMENTS'], dtype='object')

In [13]:
df[df.isnull().any(axis = 1)].shape

(314, 17)

Imputamos a 0 los valores nulos, así podemos ver si los elementos anómalos son aquellos que tienen estas columnas a 0

In [14]:
df = df.fillna(0)

Hemos aprendido el algoritmo DBSCAN, lo bueno de usar este algoritmo para detección de anomalías es que no asigna un cluster a todos los puntos, sino aquellos puntos que están más separados del resto se etiquetan automáticamente como valores extremos.

In [15]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

Al usar un algoritmo de clustering basado en densidad tenemos que estandarizar los datos.

In [16]:
df_normalizado = pd.DataFrame(StandardScaler().fit_transform(df))

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [17]:
clusterer = DBSCAN()
cluster_labels = clusterer.fit_predict(df_normalizado)
pd.Series(cluster_labels).value_counts()

-1     6627
 0     1948
 10      60
 2       34
 15      30
 7       23
 14      23
 8       14
 6       13
 3       11
 29      10
 21       9
 5        9
 9        8
 12       8
 1        8
 26       8
 17       7
 11       7
 27       7
 19       7
 4        6
 13       6
 23       6
 28       5
 30       5
 16       5
 24       5
 22       5
 25       5
 20       5
 18       5
 35       5
 31       5
 32       4
 34       4
 33       3
dtype: int64

Vemos que por defecto DBSCAN produce demasiados outliers (más de 7000 elementos anómalos del total de 9000 transacciones. Podemos usar el coeficiente de silueta (`silhouette_score`) para ver como separa el algoritmo a los buenos clientes de los (potencialmente) malos.

In [18]:
from sklearn.metrics import silhouette_score

In [19]:
silhouette_score(df_normalizado, cluster_labels)

-0.46596190778573116

Un coeficiente de silueta negativo es bastante malo. Podemos hacer una búsqueda aleatoria para optimizar dicho resultado.

In [20]:
DBSCAN().get_params()

{'algorithm': 'auto',
 'eps': 0.5,
 'leaf_size': 30,
 'metric': 'euclidean',
 'metric_params': None,
 'min_samples': 5,
 'n_jobs': None,
 'p': None}

In [22]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform 


distribucion_parametros = {
    "eps": uniform(0,5),
    "min_samples": sp_randint(2, 20),
    "p": sp_randint(1, 3),
}

In [23]:
distribucion_parametros

{'eps': <scipy.stats._distn_infrastructure.rv_frozen at 0x1bd6f8f1748>,
 'min_samples': <scipy.stats._distn_infrastructure.rv_frozen at 0x1bd6f8f1be0>,
 'p': <scipy.stats._distn_infrastructure.rv_frozen at 0x1bd6f8f1940>}

Un problema que hay con el algoritmo HDBSCAN (o DBSCAN) es que no tiene el método `predict`, por lo tanto no podemos usar el método de busqueda aleatoria (RandomSearchCV) de scikit-learn.

Sin embargo, podemos desarrollar nuestro propio método de búsqueda con  [`ParameterSampler`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html), que es lo que usa scikit-learn para tomar muestras del diccionario de búsqueda de hiperparámetros.

**Este paso tarda tiempo en ejecutarse!**

In [33]:
import numpy as np
from sklearn.model_selection import ParameterSampler

n_muestras = 100 # probamos 20 combinaciones de hiperparámetros
n_iteraciones = 3 #para validar, vamos a entrenar para cada selección de hiperparámetros en 3 muestras distintas
pct_muestra = 0.7 # usamos el 70% de los datos para entrenar el modelo en cada iteracion
resultados_busqueda = []
lista_parametros = list(ParameterSampler(distribucion_parametros, n_iter=n_muestras))

for param in lista_parametros:
    for iteration in range(n_iteraciones):
        param_resultados = []
        muestra = df_normalizado.sample(frac=pct_muestra)
        etiquetas_clusters = DBSCAN(n_jobs=-1, **param).fit_predict(muestra)
        try:
            param_resultados.append(silhouette_score(muestra, etiquetas_clusters))
        except ValueError: # a veces silhouette_score falla en los casos en los que solo hay 1 cluster
            pass
    puntuacion_media = np.mean(param_resultados)
    resultados_busqueda.append([puntuacion_media, param])

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid

In [34]:
sorted(resultados_busqueda, key=lambda x: x[0], reverse=True)[:5]

[[0.749018278617725, {'eps': 4.745213366201398, 'min_samples': 10, 'p': 1}],
 [0.741716474106214, {'eps': 4.981698293453857, 'min_samples': 14, 'p': 2}],
 [0.7416284232855861, {'eps': 4.782283682818845, 'min_samples': 13, 'p': 2}],
 [0.7335724628139673, {'eps': 4.673557587098347, 'min_samples': 5, 'p': 2}],
 [0.7327615465034267, {'eps': 4.678605084801343, 'min_samples': 9, 'p': 2}]]

In [35]:
mejores_params = {'eps': 4.745213366201398, 'min_samples':10, 'p': 1}

clusterer = DBSCAN(n_jobs=-1, **mejores_params)

etiquetas_cluster = clusterer.fit_predict(df_normalizado)

In [36]:
pd.Series(etiquetas_cluster).value_counts()

 0    8879
-1      71
dtype: int64

Vemos que hay 60 anomalias potenciales.

In [37]:
def resumen_cluster(cluster_id):
    cluster = df[etiquetas_cluster == cluster_id]
    resumen_cluster = cluster.mean().to_dict()
    resumen_cluster["cluster_id"] = cluster_id
    return resumen_cluster

def comparar_clusters(*cluster_ids):
    resumenes = []
    for cluster_id in cluster_ids:
        resumenes.append(resumen_cluster(cluster_id))
    return pd.DataFrame(resumenes).set_index("cluster_id").T

In [39]:
comparar_clusters(0, -1)

cluster_id,0,-1
BALANCE,1526.628065,6297.452338
BALANCE_FREQUENCY,0.876657,0.954076
CASH_ADVANCE,932.764611,6744.781376
CASH_ADVANCE_FREQUENCY,0.133753,0.309091
CASH_ADVANCE_TRX,3.119833,19.380282
CREDIT_LIMIT,4423.147667,13347.887324
INSTALLMENTS_PURCHASES,380.104829,4283.163944
MINIMUM_PAYMENTS,774.430432,8281.466145
ONEOFF_PURCHASES,527.734622,8683.926197
ONEOFF_PURCHASES_FREQUENCY,0.200019,0.507387
